In [33]:
#This file convert the pubmed QA dataset (https://andy-jqa.github.io/doc/PubMedQA.pdf) 
#into SQUAD format for yes/no type questions which can be used for training the model.  
import json

In [50]:
class ProcessPubmed:
    def  __init__(self,data_file_path,output_file):
        self.loadFromJson(data_file_path)
        self.yes_no_data = {"version":"BioASQ8b","data":[{"title":"PUBMED","paragraphs":[]}]}
        self.single_example = {"context":"","qas":[{"question":"","id":"","answers":"","is_impossible":False}]}
        self.output_file = output_file
        
    def loadFromJson(self,data_file):
        with open(data_file, 'r') as f:
            datastore = json.load(f)
        self.datastore = datastore
        return datastore
    
    def createQuestionAnswerPair(self,question,context,label,answer,qid):
        self.single_example = {"context":"","qas":[{"question":"","id":"","answers":"","is_impossible":False}]}
        self.single_example["context"] =context
        self.single_example["qas"][0]["question"] = question
        self.single_example["qas"][0]["id"] = qid
        self.single_example["qas"][0]["answers"] = answer
        if answer=="yes":
            print("yes")
            self.single_example["qas"][0]["is_impossible"] = False
        else:
            self.single_example["qas"][0]["is_impossible"] = True
        self.yes_no_data["data"][0]["paragraphs"].append(self.single_example)
        
    def ProcessEachQuestion(self,ques_data,qid):
        #Following are the cols available "QUESTION","CONTEXTS" [],"LABELS" [],"MESHES"[] ,"LONG_ANSWER","final_decision"
        if ques_data["final_decision"]=="maybe":
            return
        for i in range(0,len(ques_data["LABELS"])):
            if i<10:
                ques_id = qid+qid+qid+"_00"+str(i)
            else:
                ques_id = qid+qid+qid+"_0"+str(i)            
            self.createQuestionAnswerPair(ques_data["QUESTION"],ques_data["CONTEXTS"][i],ques_data["LABELS"][i],ques_data["final_decision"],ques_id)

    def ProcessEntireData(self):
        for data in self.datastore:
            self.ProcessEachQuestion(self.datastore[data],data)
        self.WriteCSVFile()
        
    def WriteCSVFile(self):
        with open(self.output_file, 'w+', encoding='utf-8') as outfile:
            json.dump(self.yes_no_data, outfile,indent=5)

In [ ]:
process_pm = ProcessPubmed("../Dataset/Pubmed/ori_pqal.json","../Dataset/Pubmed/pubmed_squad_format.json")
process_pm.ProcessEntireData()